In [ ]:
import mne
import mne_bids
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt

from nilearn import *

# Setup Paths

## Set some generic BIDS information

In [ ]:
#Raw data
bids_root = op.join('/data/', os.environ['USER'], 'meg_workshop_data')  
#Processed Data Folder: contains (freesurfer / Day1 / Day2) 
deriv_root = op.join(bids_root, 'derivatives')
if not op.exists(deriv_root): os.mkdir(deriv_root)
#This course data - bids derivatives outputs for Day2 - hey thats today
project_root = op.join(deriv_root, 'Day1')  
#Freesurfer brain surface reconstruction
fs_subjects_dir = op.join(deriv_root, 'freesurfer','subjects')
subjects_dir = fs_subjects_dir

### Find data for a subject

In [ ]:
subject = 'ON02811'
bids_path = mne_bids.BIDSPath(root=bids_root, subject = subject, session='01', task='airpuff', run='01', datatype='meg')
project_path = bids_path.copy().update(root=project_root, check=False)
if not op.exists(project_path.root): os.mkdir(project_path.root)

t1_bids_path = bids_path.copy().update(datatype='anat', session='01', task=None, run=None, suffix='T1w', acquisition='MPRAGE', extension='.nii.gz')

In [ ]:
raw = mne_bids.read_raw_bids(bids_path, verbose=False)

In [ ]:
trans = mne_bids.get_head_mri_trans(bids_path, t1_bids_path=t1_bids_path, fs_subject='sub-'+subject, fs_subjects_dir=subjects_dir)

## Transform is a 3x3 rotation matrix + the last column is the x/y/z translation

In [ ]:
trans

### Prepare Imaging for Source Localization

In [ ]:
# fwd_name=f'{output_dir}/{subjid}-fwd.fif'
fwd_name = project_path.copy().update(extension='.fif', suffix='fwd')

In [ ]:
subjects_dir

## Run the following cell and copy the output line into the terminal.<br>  This will launch Afni and open the subject file for localizing fiducials.

In [ ]:
print(f'afni {t1_bids_path.fpath}') 

In [ ]:
!afni /data/stoutjd/meg_workshop_data/sub-ON02811/ses-01/anat/sub-ON02811_ses-01_acq-MPRAGE_T1w.nii.gz

### Plot the transform

In [ ]:
project_path.directory.mkdir(parents=True, exist_ok=True)

In [ ]:
%matplotlib qt
tmp = mne.viz.plot_alignment(raw.info, trans=trans, subject='sub-'+subject, subjects_dir=subjects_dir, dig=True)

In [ ]:
# fwd_name=f'{output_dir}/{subjid}_vol-fwd.fif'
vol_fwd_fname = project_path.copy().update(extension='.fif', suffix='volfwd')
bemsol_fname = vol_fwd_fname.copy().update(suffix='bemsol')
if not os.path.exists(vol_fwd_fname):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    # mne.bem.make_watershed_bem('sub-'+subject, subjects_dir=fs_subjects_dir)
    bem = mne.make_bem_model('sub-'+subject, subjects_dir=fs_subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    mne.bem.write_bem_solution(bemsol_fname.fpath, bem_sol, overwrite=True)
    src = mne.source_space.setup_volume_source_space(subject='sub-'+subject, subjects_dir=fs_subjects_dir,
                                                     mri='T1.mgz', bem=bem_sol) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(vol_fwd_fname, forward, overwrite=True)
else:
    forward = mne.read_forward_solution(vol_fwd_fname)

## Volumetric Source Model 

In [ ]:
src = forward['src']
src.plot(subject, subjects_dir=fs_subjects_dir, brain=False, trans=trans)

### Surface Source

In [ ]:
# fwd_name=f'{output_dir}/{subjid}-fwd.fif'
fwd_fname = project_path.copy().update(extension='.fif', suffix='fwd')

if not os.path.exists(fwd_name):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    bem = mne.make_bem_model('sub-'+subject, subjects_dir=subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    src = mne.source_space.setup_source_space(subject='sub-'+subject, subjects_dir=subjects_dir, n_jobs=-1) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(fwd_name, forward)
else:
    forward = mne.read_forward_solution(fwd_name)

In [ ]:
src = forward['src']
src.plot(subjects_dir=subjects_dir, head=False, brain='white', trans=trans)

## Boundary Element Model (For MEG - red boundary in figure is all that is necessary)

In [ ]:
%matplotlib inline
_ = mne.viz.plot_bem(subject='sub-'+subject, subjects_dir=fs_subjects_dir)

## Spatial relationship of the brain to the MEG sensors

In [ ]:
mne.viz.plot_alignment(raw.info, trans=trans, subject='sub-'+subject, subjects_dir=fs_subjects_dir, dig=True, surfaces='pial')

## What is a forward Model?
## Simulation of all the source locations through the BEM

In [ ]:
print(forward['sol']['data'].shape)
print('Channels, Source Locations X 3')

In [ ]:
from mne.viz.topomap import plot_topomap

In [ ]:
tmp=raw.copy()
grads = [i for i in tmp.ch_names if i[0]=='M']
_=tmp.pick_channels(grads)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(3,2)
axs[0,0].set_title('Location 1 Forward Model')
plot_topomap(forward['sol']['data'][:,1003], tmp.info, axes=axs[0,0])
plot_topomap(forward['sol']['data'][:,1004], tmp.info, axes=axs[1,0])
plot_topomap(forward['sol']['data'][:,1005], tmp.info, axes=axs[2,0])

axs[0,1].set_title('Location 2 Forward Model')
plot_topomap(forward['sol']['data'][:,5003], tmp.info, axes=axs[0,1])
plot_topomap(forward['sol']['data'][:,5004], tmp.info, axes=axs[1,1])
plot_topomap(forward['sol']['data'][:,5005], tmp.info, axes=axs[2,1])


In [ ]:
%matplotlib inline
fig=plt.figure(figsize=(12,8))
plt.imshow(forward['sol']['data'], aspect='auto', cmap='jet', vmax=3e-6, vmin=-3e-6)
plt.title('Forward Model Simulation at every point in the source model')
plt.xlabel('Source Location')
plt.ylabel('Channel index')